In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

train = pd.read_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Inputs/train_imputed.csv')
test=pd.read_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Inputs/test_imputed.csv')
test_id=pd.read_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Inputs/test_imputed.csv')
print(train.head(5))
print(test.head(5))
train.dtypes

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
#fmri = sns.load_dataset("fmri")
sns.set(font_scale = 4)
plt.rcParams["figure.figsize"] = (50, 30)
ax = sns.lineplot(x="Day_No", y="Sales", data=train)
#hue="Course_Domain",

In [ ]:
train_chart = train[train['Day_No']<120]
train_chart.head()

sns.set(font_scale = 4)
plt.rcParams["figure.figsize"] = (50, 30)
#plt.rcParams["axes.labelsize"] = 100
ax = sns.lineplot(x="Day_No", y="Sales", data=train_chart)
#hue="Course_Domain",
#hue="Course_Domain",

In [2]:
import datetime
def serial(srl_no):
    new_date = datetime.datetime(1970,1,1,0,0) + datetime.timedelta(srl_no - 1)
    return new_date.strftime("%Y-%m-%d")

#import datetime
train_date = train
train_date['date']= train_date['Day_No'].apply(lambda x: serial(x))
train_date['date'] = pd.to_datetime(train_date['date'])
train_date.set_index('date',inplace=True)

test_date=test
test_date['date']= test_date['Day_No'].apply(lambda x: serial(x))
test_date['date'] = pd.to_datetime(test_date['date'])
test_date.set_index('date',inplace=True)


#serial_date_to_string(train_date['Day_No'])


In [3]:
test_date
#train_train_filtered=train_date[train_date['Day_No']<801]
#train_filtered.head(802)
#train_val_filtered=train_date[train_date['Day_No']>800]
#train_val_filtered

,ID,Day_No,Course_ID,Course_Domain,Course_Type,Short_Promotion,Public_Holiday,Long_Promotion,Competition_Metric
date,,,,,,,,,
1972-06-01,883,883,1,Development,Course,1,0,1,0.007
1972-06-02,884,884,1,Development,Course,1,0,1,0.007
1972-06-03,885,885,1,Development,Course,1,0,1,0.007
1972-06-04,886,886,1,Development,Course,1,0,1,0.007
1972-06-05,887,887,1,Development,Course,0,0,1,0.007
...,...,...,...,...,...,...,...,...,...
1972-07-26,548083,938,600,Software Marketing,Program,1,0,1,0.070
1972-07-27,548084,939,600,Software Marketing,Program,1,0,1,0.070
1972-07-28,548085,940,600,Software Marketing,Program,1,0,1,0.070


In [4]:
##for testing different models against validatin dataset (day 801 to 883)

from pmdarima import auto_arima
final_table= pd.DataFrame(columns=['Course_ID', 'Sales', 'Day_No'])
Day_No=range(801,883)
train_date_filtered=train_date[train_date['Day_No']<801]
val_date=train_date[train_date['Day_No']>800]

for ID in range(1,6):
    train_train=train_date_filtered[train_date['Course_ID']==ID]
    train_exog=train_train[['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
    train_endog=train_train[['Sales']]
    
        
    #for x in (7,14,15,28,30):
    model = auto_arima(train_endog,exogenous=train_exog,start_p=1,start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    #
    print(model.aic())
    print(ID)
    model.fit(train_endog)
    
    
    
    #test_test=test_date[test_date['Course_ID']==ID]    
    #train_train=train_train_filtered[train_train_filtered['Course_ID']==ID]
    val_exog=val_date.loc[val_date['Course_ID']==ID,['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
    #val_exog=train_val_filtered[train_train_filtered['Course_ID']==ID],['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
    forecast = model.predict(n_periods=len(val_exog),exogenous=val_exog)
    
    #,exog=val_exog
    #forecast
    
    #print(forecast)
    s=pd.DataFrame({'Course_ID':ID,'Sales':forecast,'Day_No':Day_No,'m':7})
    #print(s)
    final_table=final_table.append(s,ignore_index=True)

final_table.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Outputs/Arima_testing_withexog_m7_course(1-5)attempt2.csv',index=False)


C:\Users\np8022\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.


ValueError: cannot reindex from a duplicate axis

In [ ]:
##Surprisingly, adding exogenous variables brings down the prediction accuracy as revealed in a test for 5 courses 
##actual prediction for test data


from pmdarima import auto_arima
final_table= pd.DataFrame(columns=['Course_ID', 'Sales', 'Day_No'])
Day_No=range(883,943)

for ID in range(176,601):
    train_train=train_date[train_date['Course_ID']==ID]
    train_exog=train_train[['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
    train_endog=train_train[['Sales']]
    
        
    #for x in (7,14,15,28,30):
    model = auto_arima(train_endog,exogenous=train_exog,start_p=1,start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    #
    print(model.aic())
    print(ID)
    model.fit(train_endog)
    
    
    #test_test=test_date[test_date['Course_ID']==ID]    
    #train_train=train_train_filtered[train_train_filtered['Course_ID']==ID]
    test_exog=test_date.loc[test_date['Course_ID']==ID,['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
    #val_exog=train_val_filtered[train_train_filtered['Course_ID']==ID],['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
    forecast = model.predict(n_periods=len(test_exog),exogenous=test_exog)
    
    #,exog=val_exog
    #forecast
    
    #print(forecast)
    s=pd.DataFrame({'Course_ID':ID,'Sales':forecast,'Day_No':Day_No,'m':7})
    #print(s)
    final_table=final_table.append(s,ignore_index=True)

final_table.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Outputs/Arima_forecasting_withexog_m7_course(176-600).csv',index=False)


In [ ]:
#val_exog=train_val_filtered.loc[train_val_filtered['Course_ID']==1,['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
#train_val_filtered[train_val_filtered['Course_ID']==1].[train_val_filtered[['Short_Promotion','Public_Holiday','Long_Promotion','Competition_Metric']]
#val_exog
final_table
#final_table.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Outputs/Arima_forecasting_withexog_m7_course(1-174).csv',index=False)

#train_exog
#final_table.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Outputs/Arima_testing_exog_m7_course1.csv',index=False)


In [ ]:
#warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font='IPAGothic')
import numpy as np
import statsmodels.api as sm

In [ ]:
res = sm.tsa.seasonal_decompose(train_time,freq=7)
fig = res.plot()
fig.set_figheight(8)
fig.set_figwidth(15)
plt.show()

In [ ]:
from pmdarima import auto_arima
stepwise_model = auto_arima(train_time, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=15,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
#print(stepwise_model.aic())

stepwise_model.fit(train_model)
future_forecast = stepwise_model.predict(n_periods=96)

In [ ]:
#load the data
#data = pd.read_csv('international-airline-passengers.csv')

#divide into train and validation set
train = train_time[:int(0.7*(len(train_time)))]
valid = train_time[int(0.7*(len(train_time))):]

#preprocessing (since arima takes univariate series as input)
#train.drop('Month',axis=1,inplace=True)
#valid.drop('Month',axis=1,inplace=True)

#plotting the data
#train['International airline passengers'].plot()
#valid['International airline passengers'].plot()

In [ ]:
from pmdarima import auto_arima
model = auto_arima(train, trace=True, error_action='ignore', suppress_warnings=True)
model.fit(train)

forecast = model.predict(n_periods=len(valid))
forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

#plot the predictions for validation set
plt.plot(train, label='Train')
plt.plot(valid, label='Valid')
plt.plot(forecast, label='Prediction')

sns.set(font_scale = 1)
plt.rcParams["figure.figsize"] = (100, 10)

plt.show()


In [ ]:
list = ['ID','Course_ID']

#promotion and holiday flags to be tried as objects seperately

for x in train.columns:
    if x in list:
        train[x]=train[x].astype(np.object)        
    
for x in test.columns:
    if x in list:
        test[x]=test[x].astype(np.object)

test=test.drop(['ID'],axis=1)
train = train.drop(['ID','User_Traffic'],axis=1)

print(test.dtypes)

print(train.shape)
print(test.shape)


In [ ]:
train = pd.get_dummies(train, columns=['Course_ID','Course_Domain','Course_Type'],drop_first=True)
test = pd.get_dummies(test, columns=['Course_ID','Course_Domain','Course_Type'],drop_first=True)

print(train.shape)
print(test.shape)

In [ ]:
import seaborn as sns
sns.set(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x='Competition_Metric', y="Sales", data=train)

plt.rcParams["figure.figsize"] = (20, 10)
#plt.rcParams["xtick.labelsize"] = 20
#plt.autofmt_x()

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
#fmri = sns.load_dataset("fmri")
plt.rcParams["figure.figsize"] = (20, 10)
ax = sns.lineplot(x="User_Traffic", y="Sales", data=train)

In [ ]:
#Alternate code for grodsearch

from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import fbeta_score, make_scorer
MSE = make_scorer(mean_squared_error, greater_is_better=False)

#RMSE = sqrt(mean_squared_error())

from sklearn.metrics import mean_squared_log_error
#RMSLE=np.sqrt(mean_squared_log_error( y, predictions ))


In [ ]:
#Alternate code for gridsearch with stratkfold\n",
X,y=train.drop(['Sales'],axis=1),train.Sales
params = {'n_estimators': [600,1000],
        'reg_lambda' : [4,9],
        'learning_rate' : [0.03,0.13],
        'depth' : [4,7],
        'bagging_temperature' : [1]
        }                        
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.model_selection import KFold
kf = KFold(n_splits=2, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV

categorical_features_indices = np.where(X.dtypes =='object')[0]
catb = CatBoostRegressor(cat_features=categorical_features_indices,random_seed=13,early_stopping_rounds=100)

skf = StratifiedKFold(n_splits=3, shuffle = True, random_state = 13)
print('start_gridsearch')
grid = GridSearchCV(estimator=catb, param_grid=params, cv=skf.split(X,y),verbose=100,scoring = MSE )
print('start_fit')
grid.fit(X, y)

results = pd.DataFrame(grid.cv_results_)

results.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Outputs/Catreg_tuning_v1', index=False)

In [ ]:
#Catboost without gridsearch
    
from catboost import CatBoostClassifier,Pool, cv,CatBoostRegressor
from sklearn.model_selection import KFold,train_test_split
X,y=train.drop(['Sales'],axis=1),train.Sales
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.25,random_state = 13)
categorical_features_indices = np.where(X_train.dtypes =='object')[0]
err=[]
y_pred_tot=[]
cat_model5 = CatBoostRegressor(n_estimators=1200, # use large n_estimators deliberately to make use of the early stopping
                         reg_lambda=4,
                         eval_metric='RMSE',
                         random_seed=13,
                         learning_rate = 0.13,
                         depth = 7,
                         bagging_temperature = 1.0,cat_features=categorical_features_indices)

cat_model5.fit(X_train.values,y_train.values,eval_set=(X_val, y_val),plot=False,use_best_model=True,verbose_eval=50)
pred = cat_model5.predict(test)
y_pred_tot.append(pred)

In [ ]:
# y_pred=rf.predict(dftest_dum.drop('id',axis=1))
print(np.mean(err,0))
y_pred=np.mean(y_pred_tot,0)

In [ ]:
s=pd.DataFrame({'ID':test_id.ID,'Sales':y_pred})
s.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/Womens Hack/Outputs/Catreg_v0.1(RMSE for tuning_flagsasobjects).csv',index=False)
s.head()

In [ ]:
train_chart=train
train_chart['year_month']=train_chart['Year'].astype(str)+train_chart['Month'].astype(str)
print(train_chart.head())
print(train_chart.describe())
import seaborn as sns
sns.set(style="whitegrid")
tips = sns.load_dataset("tips")
ax = sns.barplot(x='year_month', y="case_count", data=train_chart)

In [ ]:
train_chart_line=train.groupby(['application_date'])['case_count'].sum().reset_index()
train_chart_line.head()

In [ ]:
sns.set(style='darkgrid')
sns.lineplot(x='application_date', y='case_count', data=train_chart_line)
plt.figure(figsize=(8,4))
plt.rcParams["figure.figsize"] = (50, 6)
plt.rcParams["xtick.labelsize"] = 7

In [ ]:
test_id=test

test_id.shape

In [ ]:
test.drop('application_date',axis=1,inplace=True)

In [ ]:
def mape(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)

    error = (predict - actual)/actual
    abs_error = np.absolute(error)
    
    #square_distance = distance ** 2

    #mean_square_distance = square_distance.mean()

    score = np.mean(abs_error)

    return score

from sklearn.metrics import make_scorer
mape_score = make_scorer(mape,greater_is_better = False)

#gsSVR = GridSearchCV(...scoring=rmse_score...)
#gsSVR.fit(X_train,Y_train)
#SVR_best = gsSVR.best_estimator_
#print(gsSVR.best_score_)

In [ ]:
#Alternate code for gridsearch/tuning

# A parameter grid for XGBoost
params = {
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.4, 0.8,1],
        'max_depth': [ 5, 8,10],
        'learning_rate':[0.01,0.1,0.3],
        'n_estimators':[30,50,80]
        }



Xd,yd=train.drop('case_count',axis=1),train.case_count
from sklearn.metrics import mean_squared_error
import xgboost as xgb
xg = xgb.XGBRegressor(eval_metric='auc',verbose=100, nthread=4)          

from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3, random_state=13, shuffle=True)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=xg, param_grid=params, n_jobs=4,scoring=mape_score, cv=skf.split(Xd,yd), verbose=100 )
grid.fit(Xd, yd)
#print('\n All results:')
#print(grid.cv_results_)
#print('\n Best estimator:')
#print(grid.best_estimator_)
#print('\n Best score:')
#print(grid.best_score_ * 2 - 1)
#print('\n Best parameters:')
#print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
 
results.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/LnTFS/Outputs/XGB_tuning_aggregated_v0.3_saved.csv', index=False)
#y_test = grid.best_estimator_.predict_proba(test)
#results_df = pd.DataFrame(data={'id':test_df['id'], 'target':y_test[:,1]})
#results_df.to_csv('submission-grid-search-xgb-porto-01.csv', index=False)

In [ ]:
#XGBoost code,using tuned models to predict

X,y=train.drop('case_count',axis=1),train.case_count

from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error

import xgboost as xgb
xg_reg = xgb.XGBRegressor(eval_metric='mae',learning_rate = 0.1,
                max_depth = 10,colsample_bytree=1, subsample=0.6, n_estimators = 50,verbose=50)

err=[]
y_pred_tot=[]
from sklearn.model_selection import KFold,StratifiedKFold
#fold=StratifiedKFold(n_splits=5,shuffle=True,random_state=1994)

#for train_index, test_index in fold.split(X,y):
#    X_train, X_test = Xd.iloc[train_index], Xd.iloc[test_index]
#    y_train, y_test = yd[train_index], yd[test_index]
print('start')
xg_reg.fit(X,y)
#    print('pred...')
#    p=xg_reg.predict(X_test)
#    print("err: ",100*np.sqrt(mean_squared_error(y_test,p)))
#    err.append(100*np.sqrt(mean_squared_error(y_test,p)))
pred = xg_reg.predict(test)
y_pred_tot.append(pred)

In [ ]:
# y_pred=rf.predict(dftest_dum.drop('id',axis=1))
print(np.mean(err,0))
y_pred=np.mean(y_pred_tot,0)
print(y_pred)

s=pd.DataFrame({'case_count':y_pred})
s.to_csv('C:/Users/np8022/Desktop/Analytics Vidhya competition/LnTFS/Outputs/XGB_Aggregated_Tuned(MAPE)_0.4_saved.csv',index=False)
s.head()